This algorithm aims to compute the evolutionary strategy of stay-and-help of our model. To do so, we evaluate the inclusive fitness change when we increase slightly the level of stay-and-help. If this change is positive, we increment level of help by a small amount. We do this process for the level of stay-and-help in low-quality group h_x and high quality groups h_y. When the inclusive change of both stay-and-help strategies are in a tolerance of 0, we assume we found the evolutionary equilibrium (h_x,h_y).

# Getting started: libraries, parameters and functions

In [1]:
#Libraries
using LinearAlgebra
using Plots
using LaTeXStrings
using Profile
using Random
using CSV
using DataFrames

In [2]:
#Parameters

#survival
s_u = 0.5
s_x = 0.5
s_y = 0.5
s_ax = 0.5
s_ay = 0.5

#fecundity
b_x = 3
b_y = 3

phi = 0.1
M = 1
#Male and female reproduction:
theta_x(x,y) = (1-phi)*x/(x+y)
theta_y(x,y) = (1-phi)*y/(x+y)
psi_x(x,y) = 1+phi+theta_x(x,y)
psi_y(x,y) = 1+phi+theta_y(x,y)

#competition
p_u = 0.5 #floater ability
p_a = 0.25 #helper ability
a = 2 #strength of the density
#Actual probability to find a breeding spot during a time step
p_u_t(x,y) = p_u/(1+a*(x+y))
p_a_t(x,y) = p_a/(1+a*(x+y))

#Cooperation levels
h_x = 0
h_y = 0
h_x_bar = 0
h_y_bar = 0

#Parameter dictionary
parameters = Dict("b_x"=>b_x,"b_y"=>b_y,"s_u"=>s_u,"s_ax"=>s_ax,"s_ay"=>s_ay,"s_x"=>s_x,"s_y"=>s_y,"phi"=>phi,"M"=>M,"a"=>a,"p_u"=>p_u,"p_a"=>p_a,"h_x" =>h_x, "h_y" =>h_y,"h_x_bar" =>h_x_bar, "h_y_bar" =>h_y_bar)


Dict{String, Real} with 16 entries:
  "s_u"     => 0.5
  "h_x_bar" => 0
  "h_x"     => 0
  "p_u"     => 0.5
  "s_y"     => 0.5
  "a"       => 2
  "h_y_bar" => 0
  "M"       => 1
  "phi"     => 0.1
  "s_ay"    => 0.5
  "b_x"     => 3
  "h_y"     => 0
  "b_y"     => 3
  "s_x"     => 0.5
  "p_a"     => 0.25
  "s_ax"    => 0.5

Usefull functions: Wild type population matrix A, mutant population type matrix M, computations of the eigenvectors mu (right) and nu (left) and relatedness coefficient.

In [3]:
#Population matrix
function A(x,y,parameters) #x and y are the number of breeders in low quality (x) and high quality (y) groups
    #First we retrieve the parameters from the dictionnary parameters
    b_x = parameters["b_x"]
    b_y = parameters["b_y"]
    s_u = parameters["s_u"]
    s_ax = parameters["s_ax"]
    s_ay = parameters["s_ay"]
    s_x = parameters["s_x"]
    s_y = parameters["s_y"]
    p_u = parameters["p_u"]
    p_a = parameters["p_a"]
    a = parameters["a"]
    M = parameters["M"]
    phi = parameters["phi"]
    h_x = parameters["h_x"]
    h_y = parameters["h_y"]
    h_x_bar = parameters["h_x_bar"]
    h_y_bar = parameters["h_y_bar"]
    
    #We use the parameters to compute the functions
    p_u_t(x,y) = p_u/(1+a*(x+y))
    p_a_t(x,y) = p_a/(1+a*(x+y))
    theta_x(x,y) = (1-phi)*x/(x+y)
    theta_y(x,y) = (1-phi)*y/(x+y)
    psi_x(x,y) = theta_x(x,y)+1+phi
    psi_y(x,y) = theta_y(x,y)+1+phi
    T_x(h_x) = 1-exp(-h_x*b_x)
    T_y(h_y) = exp(-h_y*b_y)
    
    #Finally, we return the matrix A(x,y):
    return [s_u*(1-p_u_t(x,y)) 0 0 (1-h_x)*s_u*(1-p_u_t(x,y))+h_x*(T_x(h_x_bar)*s_ay+(1-T_x(h_x_bar))*s_ax)*(1-p_a_t(x,y)) (1-h_y)*s_u*(1-p_u_t(x,y))+h_y*(T_y(h_y_bar)*s_ax+(1-T_y(h_y_bar))*s_ay)*(1-p_a_t(x,y));
    s_u*p_u_t(x,y) (1-T_x(h_x_bar))*s_x T_y(h_y_bar)*s_x (1-h_x)*s_u*p_u_t(x,y)+h_x*(T_x(h_x_bar)*s_ay+(1-T_x(h_x_bar))*s_ax)*p_a_t(x,y) (1-h_y)*s_u*p_u_t(x,y)+h_y*(T_y(h_y_bar)*s_ax+(1-T_y(h_y_bar))*s_ay)*p_a_t(x,y);
    0 T_x(h_x_bar)*s_y (1-T_y(h_y_bar))*s_y 0 0
    s_u*p_u_t(x,y)*b_x (1-T_x(h_x_bar))*s_x*b_x T_y(h_y_bar)*s_x*b_x ((1-h_x)*s_u*p_u_t(x,y)+h_x*(T_x(h_x_bar)*s_ay+(1-T_x(h_x_bar))*s_ax)*p_a_t(x,y))*b_x ((1-h_y)*s_u*p_u_t(x,y)+h_y*(T_y(h_y_bar)*s_ax+(1-T_y(h_y_bar))*s_ay)*p_a_t(x,y))*b_x;
    0 T_x(h_x_bar)*s_y*b_y (1-T_y(h_y_bar))*s_y*b_y 0 0
    ]
end


function Mutant(x,y,parameters)
    #As for A, we retrieve the parameters from the dictionnary and compute the functions
    b_x = parameters["b_x"]
    b_y = parameters["b_y"]
    s_u = parameters["s_u"]
    s_ax = parameters["s_ax"]
    s_ay = parameters["s_ay"]
    s_x = parameters["s_x"]
    s_y = parameters["s_y"]
    p_u = parameters["p_u"]
    p_a = parameters["p_a"]
    a = parameters["a"]
    M = parameters["M"]
    phi = parameters["phi"]
    h_x = parameters["h_x"]
    h_y = parameters["h_y"]
    h_x_bar = parameters["h_x_bar"]
    h_y_bar = parameters["h_y_bar"]
    p_u_t(x,y) = p_u/(1+a*(x+y))
    p_a_t(x,y) = p_a/(1+a*(x+y))
    theta_x(x,y) = (1-phi)*x/(x+y)
    theta_y(x,y) = (1-phi)*y/(x+y)
    psi_x(x,y) = theta_x(x,y)+1+phi
    psi_y(x,y) = theta_y(x,y)+1+phi
    T_x(h_x) = 1-exp(-h_x*b_x)
    T_y(h_y) = exp(-h_y*b_y)
    
    #We return the value of the mutant matrix M(x,y)
    return [s_u*(1-p_u_t(x,y)) 0 0 (1-h_x)*s_u*(1-p_u_t(x,y))+h_x*(T_x(h_x_bar)*s_ay+(1-T_x(h_x_bar))*s_ax)*(1-p_a_t(x,y)) (1-h_y)*s_u*(1-p_u_t(x,y))+h_y*(T_y(h_y_bar)*s_ax+(1-T_y(h_y_bar))*s_ay)*(1-p_a_t(x,y));
    s_u*p_u_t(x,y) (1-T_x(h_x_bar))*s_x T_y(h_y_bar)*s_x (1-h_x)*s_u*p_u_t(x,y)+h_x*(T_x(h_x_bar)*s_ay+(1-T_x(h_x_bar))*s_ax)*p_a_t(x,y) (1-h_y)*s_u*p_u_t(x,y)+h_y*(T_y(h_y_bar)*s_ax+(1-T_y(h_y_bar))*s_ay)*p_a_t(x,y);
    0 T_x(h_x_bar)*s_y (1-T_y(h_y_bar))*s_y 0 0;
    s_u*p_u_t(x,y)*b_x/2*psi_x(x,y) (1-T_x(h_x_bar))*s_x*b_x/2*psi_x(x,y)+T_x(h_x_bar)*s_y*b_x/2*theta_x(x,y) T_y(h_y_bar)*s_x*b_x/2*psi_x(x,y)+(1-T_y(h_y_bar))*s_y*b_x/2*theta_x(x,y) ((1-h_x)*s_u*p_u_t(x,y)+h_x*(T_x(h_x_bar)*s_ay+(1-T_x(h_x_bar))*s_ax)*p_a_t(x,y))*b_x/2*psi_x(x,y) ((1-h_y)*s_u*p_u_t(x,y)+h_y*(T_y(h_y_bar)*s_ax+(1-T_y(h_y_bar))*s_ay)*p_a_t(x,y))*b_x/2*psi_x(x,y);
    s_u*p_u_t(x,y)*b_y/2*theta_y(x,y) T_x(h_x_bar)*s_y*b_y/2*psi_y(x,y)+(1-T_x(h_x_bar))*s_x*b_y/2*theta_y(x,y) (1-T_y(h_y_bar))*s_y*b_y/2*psi_y(x,y)+T_y(h_y_bar)*s_x*b_y/2*theta_y(x,y) ((1-h_x)*s_u*p_u_t(x,y)+h_x*(T_x(h_x_bar)*s_ay+(1-T_x(h_x_bar))*s_ax)*p_a_t(x,y))*b_y/2*theta_y(x,y) ((1-h_y)*s_u*p_u_t(x,y)+h_y*(T_y(h_y_bar)*s_ax+(1-T_y(h_y_bar))*s_ay)*p_a_t(x,y))*b_y/2*theta_y(x,y)
    ]
end



function right_eigenvector(parameters)
    #We suppose a random value for the equilibrium of the mutant matrix
    mu = rand!(zeros(5,1))
    #And compute the equilibrium value until we find a value that satisfies the tolerance (1e^-8)
    while sum(abs.(mu-A(mu[2],mu[3],parameters)*mu))>1e-8
        mu = A(mu[2],mu[3],parameters)*mu
    end
    return mu
end

function left_eigenvector(parameters,mu)
    #We suppose a random value for the left eigenvalue of the mutant matrix
    nu = rand!(zeros(1,5))
    #And compute the left eigenvector value until we find a value that satisfies the tolerance (1e^-8)
    while sum(abs.(nu-nu*Mutant(mu[2],mu[3],parameters)))>1e-8
        nu = nu*Mutant(mu[2],mu[3],parameters)
        nu /= sum(nu)
    end
    return nu
end

function R_parent(parameters)
    return 1/2*(1+parameters["phi"]) 
end

function R_siblings_x(parameters)
    b_x = parameters["b_x"] 
    M = parameters["M"]
    phi = parameters["phi"]
    if parameters["h_x"] >0 #If there are helpers:
        R_siblings_x = (1-1/(parameters["h_x"]*b_x))*(phi^2+phi*(1-phi)+(1-phi)^2*(1/4+1/(4*M)))+1/(parameters["h_x"]*b_x)
    else #If there are no helpers
        R_siblings_x = 1
    end
    return  R_siblings_x
end

function R_siblings_y(parameters)
    M = parameters["M"]
    phi = parameters["phi"]
    b_y = parameters["b_y"]
    if parameters["h_y"] >0
        R_siblings_y = (1-1/(parameters["h_y"]*b_y))*(phi^2+phi*(1-phi)+(1-phi)^2*(1/4+1/(4*M)))+1/(parameters["h_y"]*b_y)
    else
        R_siblings_y = 1
    end
    return  R_siblings_y
end

R_siblings_y (generic function with 1 method)

Now that we have our useful function, we can compute the evolution of the stay-and-help rates h_x, h_y

In [4]:
function stay_help_evol(parameters)
    #initialization
    dW_x = 1
    dW_y = 1
    signx = 1
    signy = 1
    #delta h_x, delta h_y:
    dhx = 1e-4
    dhy = 1e-4
    tol = 1e-8
    parameters["h_x"] = 0.0
    parameters["h_x_bar"] = 0.0
    parameters["h_y"] = 0.0
    parameters["h_y_bar"] = 0.0
    i = 0        
    reason = "too many iterations"

    hx_mem = 10
    hy_mem = 10
    dwx_mem = 10
    dwy_mem = 10
    #evolution
    #We loop the computations until the both inclusive fitness change
    #(Delta_h_x W and Delta_h_y W) changed their sign
    while i<100000
        reason = "too many iterations"
        
        #If we compute the same value as we found at the previous time step, we break the loop
#         if parameters["h_x"] == hx_mem && parameters["h_y"] == hy_mem
#             reason = "stagnation"
#             break
            
#         end
        #We keep track of the sign of the inclusive fitness change
        
        #Eigenvectors mu and nu
        mu = right_eigenvector(parameters)
        nu = left_eigenvector(parameters,mu)
        #h_x evolution
        parameters["h_x"] += dhx #We assume h_x is slightly higher and evaluate the change in the fitness of the individual, its breeder, the other helpers on the nest
        W_indiv_x = nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,mu[4],0]
        parameters["h_x"] -= 2*dhx
        W_indiv_x -= nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,mu[4],0]
        parameters["h_x"] += dhx

        parameters["h_x_bar"] += dhx
        W_parent_x = nu*Mutant(mu[2],mu[3],parameters)*[0,mu[2],0,0,0]
        W_sibs_x = nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,mu[4],0]

        parameters["h_x_bar"] -= 2*dhx
        W_parent_x -= nu*Mutant(mu[2],mu[3],parameters)*[0,mu[2],0,0,0]
        W_sibs_x -= nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,mu[4],0]
        parameters["h_x_bar"] += dhx

        dW_x = R_parent(parameters)*W_parent_x[1]+W_indiv_x[1]+R_siblings_x(parameters)*W_sibs_x[1]

        #h_y, same process as h_x
        parameters["h_y"] += dhy
        W_indiv_y = nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,0,mu[5]]
        parameters["h_y"] -= 2*dhy
        W_indiv_y -= nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,0,mu[5]]
        parameters["h_y"] += dhy

        parameters["h_y_bar"] += dhy
        W_parent_y = nu*Mutant(mu[2],mu[3],parameters)*[0,0,mu[3],0,0]
        W_sibs_y = nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,0,mu[5]]

        parameters["h_y_bar"] -= 2*dhy
        W_parent_y -= nu*Mutant(mu[2],mu[3],parameters)*[0,0,mu[3],0,0]
        W_sibs_y -= nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,0,mu[5]]
        parameters["h_y_bar"] += dhy

        dW_y = R_parent(parameters)*W_parent_y[1]+W_indiv_y[1]+R_siblings_y(parameters)*W_sibs_y[1]


        hx_mem = parameters["h_x"]
        hy_mem = parameters["h_y"]
        dwx_mem = dW_x
        dwy_mem = dW_y
        parameters["h_x"] += dW_x*50
        parameters["h_x_bar"] += dW_x*50
        parameters["h_y"] += dW_y*50
        parameters["h_y_bar"] += dW_y*50

        i+=1
        #We keep both values between 0 and 1 since they are probabilities
        parameters["h_x"] = clamp(parameters["h_x"],0,1)
        parameters["h_x_bar"] = clamp(parameters["h_x_bar"],0,1)
        parameters["h_y"] = clamp(parameters["h_y"],0,1)
        parameters["h_y_bar"] = clamp(parameters["h_y_bar"],0,1)
        
        #the algorithm stops if:
        #1. The inclusive fitness change is within a tolerance of 0
        if abs(dW_x)<tol && abs(dW_y)<tol
            #If we run for too long, we break the loop
            reason = "css found"
            break
        end
        #2. Both values are repeated (this can happen at the border of the [0,1]x[0,1] square)
        
        if parameters["h_x"] == hx_mem && parameters["h_y"] == hy_mem
             reason = "stagnation"
            break
        end
       
        #3. One value is repeated while the other remains an internal equilibrium: the change for both values is thus
        #null/not significant
        if parameters["h_x"] == hx_mem && abs(dW_y)<tol && abs(dwy_mem)<tol
            reason = "limit"
            break
        end
        
        if parameters["h_y"] == hy_mem && abs(dW_x)<tol && abs(dwx_mem)<tol
            reason = "limit"
            break
        end    
        
    end
    println(i)
    #Evaluation of group augmentation and cost of help at evolutionary equilibrium:
    eq = right_eigenvector(parameters)
    mu = eq
    nu = left_eigenvector(parameters,mu)
    #Group augmentation effect
    GA = (1-p_a_t(mu[2],mu[3]))*nu[1]+p_a_t(mu[2],mu[3])*(nu[2]+(1+phi)*b_x/2*nu[4]+(1-phi)*mu[2]/(mu[2]+mu[3])*b_x/2*nu[4]+(1-phi)*mu[3]/(mu[2]+mu[3])*b_y/2*nu[5])
    #Probability of establishment
    P2 = parameters["s_u"]*p_u_t(mu[2],mu[3])/(1-parameters["s_u"]*(1-p_u_t(mu[2],mu[3])))
    #cost of help
    parameters["h_x"] += dhx
    W_indiv_x = nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,mu[4],0]
    parameters["h_x"] -= 2*dhx
    W_indiv_x -= nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,mu[4],0]
    parameters["h_x"] += dhx
    cost = W_indiv_x[1]
    return parameters, P2, GA, cost, reason
end



stay_help_evol (generic function with 1 method)

Now we compute the level of stay-and-help depending on the parameters:

# Simulations

## Impact of selfing rate phi

In [ ]:
#Parameters used:
parameters["s_x"] = 0.5
parameters["s_y"] = 0.5
parameters["s_ax"] = 0.5
parameters["s_ay"] = 0.6
parameters["s_u"] = 0.5

parameters["b_x"] = 3
parameters["b_y"] = 3
parameters["phi"] = 0.1
N = 30
phi_csv = zeros(N^2)
db_csv = zeros(N^2)
hx_csv = zeros(N^2)
hy_csv = zeros(N^2)
P_csv = zeros(N^2)
parameters_csv = Array{Dict}(undef, N^2)
reason_csv = Array{String}(undef, N^2)
GA_csv = zeros(N^2)
cost_csv = zeros(N^2)
j = 1
#We loop over the parameters we are interested in, here phi and fecundity benefits:
for phi in range(0,1,N)
    parameters["phi"] = phi
    for db in range(0,2,N)
        parameters["b_y"] = parameters["b_x"]+db
        parameters, P2, GA, cost, reason = stay_help_evol(parameters)
        hx_csv[j] = parameters["h_x"]
        hy_csv[j] = parameters["h_y"]
        phi_csv[j] = phi
        db_csv[j] = db
        GA_csv[j] = GA
        cost_csv[j] = cost
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
#Finally, we write the result in a csv file
csv_data = DataFrame(db = db_csv, phi = phi_csv, h_x = hx_csv, h_y = hy_csv, cost=cost_csv, GA = GA_csv, parameters = parameters_csv, reason = reason_csv )

CSV.write("result_phi_b_cost_2.csv",csv_data)

In [6]:
csv_data = DataFrame(db = db_csv, phi = phi_csv, h_x = hx_csv, h_y = hy_csv, cost=cost_csv, GA = GA_csv, parameters = parameters_csv, reason = reason_csv )

CSV.write("result_phi_b_cost_2.csv",csv_data)

"result_phi_b_cost_2.csv"

## Help level with fecundity benefits

In [6]:
#Parameters used:
parameters["s_x"] = 0.45
parameters["s_y"] = 0.5
parameters["s_ax"] = 0.5
parameters["s_ay"] = 0.5
parameters["s_u"] = 0.45
parameters["phi"] = 0.1
N = 20
b_csv = zeros(N^2)
db_csv = zeros(N^2)
hx_csv = zeros(N^2)
hy_csv = zeros(N^2)
P2_csv = zeros(N^2)
parameters_csv = Array{Dict}(undef, N^2)
reason_csv = Array{String}(undef, N^2)
j = 1
#We loop over the parameters we are interested in, here phi and fecundity benefits:
for b in range(3,7,N)
    parameters["b_x"] = b
    for db in range(0,2,N)
        parameters["b_y"] = parameters["b_x"]+db
        parameters, P2, GA, cost, reason = stay_help_evol(parameters)
        hx_csv[j] = parameters["h_x"]
        hy_csv[j] = parameters["h_y"]
        b_csv[j] = b
        db_csv[j] = db
        P2_csv[j] = P2
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
#Finally, we write the result in a csv file
csv_data = DataFrame(db = db_csv, b = b_csv, h_x = hx_csv, h_y = hy_csv, P2 = P2_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_p_b_all_2.csv",csv_data)


1
1
1
1996
23625
29788
26445
23036
20195
17869
15950
14344
12983
11814
10800
9911
9126
8429
8136
7890
1
1
1
7428
26710
23927
20749
18123
16003
14273
12838
11629
10596
9703
8922
8234
7622
7073
6757
6566
1
1
1946
23911
22046
19035
16548
14567
12970
11657
10559
9627
8824
8125
7510
6965
6477
6037
5715
5561
1
1156
20431
20593
17729
15326
13437
11933
10709
9695
8838
8105
7468
6911
6417
5976
5580
5222
4904
4778
1
13112
19397
16728
14369
12534
11094
9935
8983
8185
7505
6918
6406
5953
5551
5190
4863
4566
4295
4155
1209
18236
15961
13618
11809
10408
9296
8390
7637
6999
6451
5975
5556
5184
4851
4551
4279
4030
3802
3649
16604
15372
13035
11225
9844
8762
7890
7172
6567
6051
5604
5212
4865
4556
4278
4026
3796
3586
3392
3232
14899
12593
10760
9381
8315
7467
6774
6196
5705
5281
4912
4586
4296
4036
3801
3587
3392
3212
3046
2892
12271
10396
9002
7941
7107
6433
5874
5403
4999
4648
4339
4066
3821
3600
3400
3217
3050
2895
2752
2618
10123
8696
7628
6800
6138
5595
5139
4751
4415
4121
3861
3629
3421
3232
3060

"result_p_b_all_2.csv"

## Help level with survival benefits and no fecundity benefits

In [5]:
#Parameters used:
parameters["b_x"] = 5
parameters["b_y"] = 5
parameters["s_u"] = 0.45
parameters["s_ax"] = 0.5
parameters["s_ay"] = 0.5
parameters["s_u"] = 0.5
parameters["phi"] = 0.1

N = 20
s_csv = zeros(N^2)
ds_csv = zeros(N^2)
hx_csv = zeros(N^2)
hy_csv = zeros(N^2)
P2_csv = zeros(N^2)
parameters_csv = Array{Dict}(undef, N^2)
reason_csv = Array{String}(undef, N^2)
j = 1
#We loop over the parameters we are interested in, here phi and fecundity benefits:
for s in range(0.2,0.7,N)
    parameters["s_x"] = s
    for ds in range(0,0.2,N)
        parameters["s_y"] = parameters["s_x"]+ds
        parameters, P2, GA, cost, reason = stay_help_evol(parameters)
        hx_csv[j] = parameters["h_x"]
        hy_csv[j] = parameters["h_y"]
        s_csv[j] = s
        ds_csv[j] = ds    
        P2_csv[j] = P2
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
#Finally, we write the result in a csv file
csv_data = DataFrame(ds = ds_csv, s = s_csv, h_x = hx_csv, h_y = hy_csv, P2 = P2_csv, parameters = parameters_csv, reason = reason_csv )

CSV.write("result_p_s_2.csv",csv_data)


1
1
1
1
1
1
1
367
9916
7666
6383
5569
5001
4578
4247
3979
3756
3566
3401
3257
1
1
1
1
1
1
1
6936
8856
6931
5840
5137
4640
4266
3971
3731
3530
3359
3210
3079
1
1
1
1
1
1
1
10459
7914
6298
5368
4757
4320
3988
3724
3508
3327
3171
3036
2916
1
1
1
1
1
1
1
9737
7103
5751
4953
4420
4034
3738
3501
3306
3142
3000
2876
2767
1
1
1
1
1
1
1
8647
6408
5273
4586
4119
3776
3511
3298
3122
2972
2843
2729
2628
1
1
1
1
1
1
4516
7649
5811
4853
4259
3848
3543
3305
3112
2952
2815
2697
2592
2499
1
1
1
1
1
1
9819
6797
5294
4482
3965
3602
3330
3116
2941
2795
2670
2561
2464
2377
1
1
1
1
1
1
8866
6078
4843
4150
3700
3378
3135
2941
2782
2649
2533
2432
2342
2261
1
1
1
1
1
1
7701
5468
4447
3853
3459
3173
2954
2779
2634
2511
2405
2310
2226
2149
1
1
1
1
1
432
6712
4948
4096
3585
3238
2984
2786
2627
2494
2381
2281
2193
2112
2038
1
1
1
1
1
9384
5902
4498
3783
3341
3035
2808
2629
2484
2361
2255
2162
2077
1999
2009
1
1
1
1
1
8176
5235
4106
3502
3118
2847
2643
2481
2347
2233
2133
2043
2000
2029
2070
1
1
1
1
1
6897
4679
376

"result_p_s_2.csv"

In [37]:

parameters["s_x"] = 0.2
parameters["s_y"] = parameters["s_x"]+0.08888888
parameters, P2, GA, cost, reason = stay_help_evol(parameters)

8.207067928626968e-6
0.0
7.98396206869862e-6
9.6636008818467e-9
7.768628769976031e-6
1.8939381653658595e-8
7.560705582045084e-6
2.7847579833593797e-8
7.359852847205199e-6
3.64070762056627e-8
7.165750700902086e-6
4.463550311290367e-8
6.978098674940987e-6
5.254934857919705e-8
6.796613562834458e-6
6.016404471315567e-8
6.621028710210312e-6
6.749405379964311e-8
6.4510920589469555e-6
7.455293707642588e-8
6.2865662386710235e-6
8.135343297667465e-8
6.12722636200991e-6
8.790750812910821e-8
5.972859984693107e-6
9.422642191574914e-8
5.823265839563499e-6
1.0032077346761045e-7
5.678253541044349e-6
1.0620055524038312e-7
5.5376421094363595e-6
1.1187518488226054e-7
5.401260110200758e-6
1.173535558992483e-7
5.268944890163275e-6
1.226440726587392e-7
5.140541608403276e-6
1.277546760421493e-7
5.015903106402045e-6
1.326928802404841e-7
4.8948894831507604e-6
1.3746580254075504e-7
4.77736745945479e-6
1.4208018531316608e-7
4.663210084406014e-6
1.465424223191271e-7
4.552296228228782e-6
1.5085857763654567e-7
4.4

1.0064105302604043e-7
2.763261176453976e-7
9.963445827821234e-8
2.7617319018494535e-7
9.864718012592144e-8
2.760197205133392e-7
9.767877959116476e-8
2.7586571678950323e-7
9.672890688844767e-8
2.75711194245177e-7
9.579723086184364e-8
2.7555616808902886e-7
9.488329122953596e-8
2.7540064285597205e-7
9.398684042843941e-8
2.752446398486717e-7
9.31074438492759e-8
2.7508816213949633e-7
9.224475243930206e-8
2.749312221518413e-7
9.139859019347403e-8
2.7477384332988755e-7
9.056844918614717e-8
2.7461602218250445e-7
8.97540373356889e-8
2.7445777078432074e-7
8.895511079548964e-8
2.7429910682864894e-7
8.817128764604387e-8
2.741400338444371e-7
8.740231308563125e-8
2.7398056555733747e-7
8.664788304774671e-8
2.7382071100291727e-7
8.590769561417762e-8
2.736604799250323e-7
8.518147901620532e-8
2.7349988146064436e-7
8.446893617758953e-8
2.7333892418414385e-7
8.37697883990254e-8
2.7317761593257903e-7
8.308379315924566e-8
2.7301596856304406e-7
8.241072951554036e-8
2.7285399340345687e-7
8.175020741602035e-8


4.33405859381951e-8
2.36046032742795e-7
4.329753663379201e-8
2.358816568022449e-7
4.3254649757795693e-8
2.3571735465856661e-7
4.321197694390482e-8
2.3555313004514425e-7
4.316948137573069e-8
2.353889805435141e-7
4.312719332905655e-8
2.352249096114133e-7
4.308497792010615e-8
2.35060905716487e-7
4.3043048778002024e-8
2.3489698708825008e-7
4.300119207933939e-8
2.3473313620781697e-7
4.295952405475854e-8
2.3456936354727961e-7
4.2917979325987556e-8
2.3440566348613437e-7
4.2876631621559214e-8
2.3424204236869826e-7
4.283540193245603e-8
2.3407849437861667e-7
4.279435378562728e-8
2.3391502521809973e-7
4.275343350735304e-8
2.3375162985176529e-7
4.271267314143179e-8
2.3358831203607061e-7
4.2672074968001697e-8
2.334250725132169e-7
4.2631590264907704e-8
2.3326190671437577e-7
4.2591204059419453e-8
2.3309881386039677e-7
4.2551027812365305e-8
2.3293580351827986e-7
4.2510936378028987e-8
2.3277286596229788e-7
4.2471006095336184e-8
2.3261000751013966e-7
4.2431190580566195e-8
2.3244722439546092e-7
4.2391507

1.9915957638347684e-7
3.523887334017095e-8
1.990143120996284e-7
3.5209386468892934e-8
1.9886912981511807e-7
3.517999037639237e-8
1.9872403492268023e-7
3.515052377639494e-8
1.985790148041104e-7
3.512112242560193e-8
1.9843408007409425e-7
3.5091768173235694e-8
1.982892294851924e-7
3.506238400175486e-8
1.9814445624760993e-7
3.503305733981992e-8
1.9799976840959724e-7
3.5003778559031186e-8
1.9785516442577533e-7
3.49744480057659e-8
1.977106364630876e-7
3.494521696595636e-8
1.9756619679999277e-7
3.4915975374860686e-8
1.974218361488686e-7
3.4886787152193935e-8
1.9727756055696113e-7
3.48576033412678e-8
1.9713336567644644e-7
3.482841394314971e-8
1.969892499523177e-7
3.4799303447187425e-8
1.9684522145069225e-7
3.477023606674037e-8
1.9670127572495414e-7
3.474107583418249e-8
1.965574044331642e-7
3.471196547005919e-8
1.9641361671037666e-7
3.468295988601211e-8
1.962699170958428e-7
3.4653928698835657e-8
1.9612629573537866e-7
3.46249652893925e-8
1.9598276043775557e-7
3.459595180749502e-8
1.9583930146675

1.6689016069087346e-7
2.8849267304234473e-8
1.6676422594247997e-7
2.882489642808128e-8
1.6663836806107158e-7
2.880056544918568e-8
1.6651259119107586e-7
2.8776241682572335e-8
1.6638689293233012e-7
2.8751953026762145e-8
1.6626127527742343e-7
2.8727668212327685e-8
1.6613573576755936e-7
2.8703416125877503e-8
1.6601027659959195e-7
2.8679137592524384e-8
1.6588489308065968e-7
2.8654941729160013e-8
1.657595939612291e-7
2.86307371824611e-8
1.6563437182049965e-7
2.8606515149742773e-8
1.6550922610266564e-7
2.858239596115604e-8
1.6538416600436443e-7
2.8558206231767103e-8
1.6525917819736786e-7
2.8534091847666444e-8
1.6513427385728505e-7
2.850996627529732e-8
1.6500944597843567e-7
2.848589628068332e-8
1.6488470058299868e-7
2.8461855147714113e-8
1.6476003483097169e-7
2.8437810338513493e-8
1.646354464084068e-7
2.8413751673730882e-8
1.6451093456833236e-7
2.8389705059028714e-8
1.6438650111373293e-7
2.8365747418991583e-8
1.6426215212733e-7
2.8341766519782087e-8
1.6413787869559385e-7
2.831780857859425e-8
1

2.3616525900166255e-8
1.3921325271396712e-7
2.3596429168941293e-8
1.3910545872426078e-7
2.3576353799792393e-8
1.3899773773900365e-7
2.3556317534095686e-8
1.3889009218420734e-7
2.35362539138833e-8
1.387825163203656e-7
2.3516229845366166e-8
1.3867501528737683e-7
2.3496241256809472e-8
1.3856758881323717e-7
2.3476241585443304e-8
1.384602330079152e-7
2.3456287802371657e-8
1.3835295294096703e-7
2.3436330072458512e-8
1.3824574428599276e-7
2.3416401147283096e-8
1.3813860955226948e-7
2.3396502937812725e-8
1.3803154861619813e-7
2.3376599175844873e-8
1.3792455904413531e-7
2.3356677578134644e-8
1.3781763999786232e-7
2.3336831853075404e-8
1.3771079799317457e-7
2.3316968672530594e-8
1.3760402640811165e-7
2.3297127284273268e-8
1.3749732819734131e-7
2.327731119661225e-8
1.3739070345592692e-7
2.325752785082007e-8
1.3728415243115252e-7
2.3237747180662673e-8
1.3717767310085192e-7
2.321792877541109e-8
1.3707126265980274e-7
2.319818963870111e-8
1.3696492935644233e-7
2.3178475960786084e-8
1.368586694879166e

1.9311651540404465e-8
1.1573597145011389e-7
1.9295172459798948e-8
1.1564456893348208e-7
1.9278662120872305e-8
1.155532286063396e-7
1.9262213998839895e-8
1.1546195695724681e-7
1.924580678575445e-8
1.153707529327929e-7
1.9229337865078744e-8
1.1527960884745828e-7
1.9212895814585277e-8
1.1518853079116337e-7
1.919652326265724e-8
1.150975221656145e-7
1.9180131055621043e-8
1.1500657647271076e-7
1.916372062011056e-8
1.1491569360524656e-7
1.9147387064766264e-8
1.1482488045415004e-7
1.9131004358464954e-8
1.1473412791672064e-7
1.9114680069322288e-8
1.1464344384154107e-7
1.909836179758977e-8
1.1455282436893844e-7
1.90820540590982e-8
1.1446226957107655e-7
1.906572816290364e-8
1.143717776988345e-7
1.904949948616921e-8
1.1428135657893732e-7
1.9033214330999026e-8
1.1419099561283189e-7
1.90169308994528e-8
1.1410069856053135e-7
1.9000683341988058e-8
1.1401046777622867e-7
1.898445208259596e-8
1.1392030213937974e-7
1.8968283879711037e-8
1.138302050724199e-7
1.8952066885295673e-8
1.1374016825899841e-7
1.89

(Dict{String, Real}("s_u" => 0.5, "h_x_bar" => 0.04036224740899632, "h_x" => 0.04036224740899633, "p_u" => 0.5, "s_y" => 0.28888888, "a" => 2, "h_y_bar" => 0.03020560999815968, "M" => 1, "phi" => 0.1, "s_ay" => 0.5…), 0.15836761721611475, 0.2309707327766029, -3.452073446386095e-5, "css found")

## Help level with group augmentation and fecundity benefits

In [17]:
#Parameters used:
parameters["s_x"] = 0.45
parameters["s_y"] = 0.5
parameters["s_ax"] = 0.5
parameters["s_ay"] = 0.6
parameters["s_u"] = 0.45
parameters["phi"] = 0.1
N = 10
b_csv = zeros(N^2)
db_csv = zeros(N^2)
hx_csv = zeros(N^2)
hy_csv = zeros(N^2)
P2_csv = zeros(N^2)
parameters_csv = Array{Dict}(undef, N^2)
reason_csv = Array{String}(undef, N^2)
j = 1
#We loop over the parameters we are interested in, here phi and fecundity benefits:
for b in range(3,7,N)
    parameters["b_x"] = b
    for db in range(0,2,N)
        parameters["b_y"] = parameters["b_x"]+db
        parameters, P2, GA, cost, reason = stay_help_evol(parameters)
        hx_csv[j] = parameters["h_x"]
        hy_csv[j] = parameters["h_y"]
        b_csv[j] = b
        db_csv[j] = db       
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
#Finally, we write the result in a csv file
csv_data = DataFrame(db = db_csv, b = b_csv, h_x = hx_csv, h_y = hy_csv, P2 = P2_csv, parameters = parameters_csv, reason = reason_csv )

CSV.write("result_p_b_GA.csv",csv_data)

"result_p_GA_2.csv"